In [1]:
import skopt
print('skopt:', skopt.__version__)

skopt: 0.10.2


In [3]:
import pandas as pd
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
df = pd.read_csv(url, header=None)

In [4]:
data = df.values
X,y = data[:,:-1], data[:,-1]
print(X.shape, y.shape)

(351, 34) (351,)


In [5]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
m_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score = 'raise')
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(m_scores), np.std(m_scores)))

Mean Accuracy: 0.937 (0.038)


In [7]:
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

search_space = list()
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='C'))
search_space.append(Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name='kernel'))
search_space.append(Integer(1, 5, name='degree'))
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='gamma'))

@use_named_args(search_space)
def evaluate_model(**params):

    model = SVC()
    model.set_params(**params)

    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    result = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    estimate = np.mean(result)
    return 1.0 - estimate

result = gp_minimize(evaluate_model, search_space)
print('Best Accuracy: %.3f' % (1.0 - result.fun)) 
print('Best Parameters: %s' % (result.x))

Best Accuracy: 0.921
Best Parameters: [0.0009988293889179134, 'poly', 2, 6.451100669436012]


In [8]:
from skopt import BayesSearchCV

params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)
search.fit(X, y)
print('Best Accuracy: %.3f' % search.best_score_)
print('Best Parameters: %s' % search.best_params_)

Best Accuracy: 0.937
Best Parameters: OrderedDict({'C': 100.0, 'degree': 2, 'gamma': 0.008083433183325325, 'kernel': 'rbf'})
